The goal of this notebook is to find the recalibrated intercept for the SBI model that reproduces the SBI rate at the Kaiser population.

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import importlib.util

from microsim.population_factory import PopulationFactory
from microsim.outcome import OutcomeType
from microsim.sbi_model import SBIModel

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)

In [2]:
pd.set_option('future.no_silent_downcasting', True) 

In [9]:
popSize = 1000000
pop = PopulationFactory.get_kaiser_population(n=popSize)
pop.advance(1, nWorkers=5)

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population_factory.py:525: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,dfForGroup])
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [10]:
#the SBI rate based on the Kaiser population, the SBI model should reproduce this rate
kaiserSbiRate = 0.0438

In [11]:
#I will use a subclass to find the recalibrated intercept instead of monkey-patching
class SBIModelRecalibrated(SBIModel):
    def __init__(self):
        pass
    
    def estimate_next_risk(self, person, interceptChange=0.):
        lp = super().calc_linear_predictor_for_patient_characteristics(
            person._gender,
            person._raceEthnicity,
            person._smokingStatus,
            person._statin[-1],
            person._afib[-1],
            person._pvd[-1],
            person._age[-1],
            person._sbp[-1],
            person._dbp[-1],
            person._bmi[-1],
            person._anyPhysicalActivity[-1],
            person._antiHypertensiveCount[-1],
            #person._otherLipidLowering,
            person._a1c[-1],
            person._totChol[-1],
            person._hdl[-1],
            person._ldl[-1],
            person._trig[-1],
            person._creatinine[-1])
        #this modifies the intercept of the model
        lp += interceptChange
        return True if person._rng.uniform()<self.inverse_logit(lp) else False

In [12]:
interceptChange=0
interceptChangeStep = 0.1
simSbiRate = np.mean(list(map(lambda x: SBIModelRecalibrated().estimate_next_risk(x, interceptChange=0), pop._people)))
diff = kaiserSbiRate-simSbiRate
if diff>0:
    interceptChange += interceptChangeStep
else:
    interceptChange += -interceptChangeStep
print(kaiserSbiRate, simSbiRate, diff, interceptChange, interceptChangeStep)

0.0438 0.055613 -0.011813000000000004 -0.1 0.1


In [13]:
for i in range(50):
    simSbiRate = np.mean(list(map(lambda x: SBIModelRecalibrated().estimate_next_risk(x, interceptChange=interceptChange), pop._people)))
    print(kaiserSbiRate, simSbiRate, diff, interceptChange, interceptChangeStep)    
    diffOld = diff
    diff = kaiserSbiRate-simSbiRate
    if diffOld*diff<0:
        interceptChangeStep = interceptChangeStep/2.
    if diff>0:
        interceptChange += interceptChangeStep
    else:
        interceptChange += -interceptChangeStep

0.0438 0.050503 -0.011813000000000004 -0.1 0.1
0.0438 0.046188 -0.006703000000000001 -0.2 0.1
0.0438 0.042248 -0.0023880000000000012 -0.30000000000000004 0.1
0.0438 0.043566 0.0015519999999999978 -0.25000000000000006 0.05
0.0438 0.045966 0.0002339999999999981 -0.20000000000000007 0.05
0.0438 0.045068 -0.0021660000000000013 -0.22500000000000006 0.025
0.0438 0.044118 -0.0012679999999999983 -0.25000000000000006 0.025
0.0438 0.04276 -0.00031799999999999884 -0.2750000000000001 0.025
0.0438 0.043454 0.0010399999999999993 -0.26250000000000007 0.0125
0.0438 0.043873 0.0003459999999999991 -0.25000000000000006 0.0125
0.0438 0.043268 -7.300000000000362e-05 -0.25625000000000003 0.00625
0.0438 0.043897 0.0005319999999999978 -0.25312500000000004 0.003125
0.0438 0.044102 -9.699999999999986e-05 -0.25468750000000007 0.0015625
0.0438 0.043591 -0.00030200000000000365 -0.2562500000000001 0.0015625
0.0438 0.044264 0.00020900000000000085 -0.2554687500000001 0.00078125
0.0438 0.043231 -0.00046399999999999914

In [14]:
print(kaiserSbiRate, simSbiRate, diff, interceptChange, interceptChangeStep)

0.0438 0.043787 1.2999999999999123e-05 -0.255443497002125 7.450580596923829e-10
